In [1]:
import requests
from bs4 import BeautifulSoup
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws&prmd=nivsbmtz&source=lnt&tbs=qdr:d&sa=X".format(ticker)
    r = requests.get(search_url)                        ###################Think how can we add second page result
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [5]:
raw_urls = search_for_stock_news_urls("TSLA")
raw_urls

['/?sa=X&ved=0ahUKEwiQ2IrtkOCGAxV5SDABHRHfCuQQOwgC',
 '/search?q=yahoo+finance+TSLA&sa=X&sca_esv=66c5d7ec1899268c&ie=UTF-8&tbs=qdr:d&tbm=nws&gbv=1&sei=jdtuZtDdOfmQwbkPkb6roA4',
 '/search?q=yahoo+finance+TSLA&sca_esv=66c5d7ec1899268c&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiQ2IrtkOCGAxV5SDABHRHfCuQQ_AUIBSgA',
 '/search?q=yahoo+finance+TSLA&sca_esv=66c5d7ec1899268c&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiQ2IrtkOCGAxV5SDABHRHfCuQQ_AUIBygC',
 '/search?q=yahoo+finance+TSLA&sca_esv=66c5d7ec1899268c&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiQ2IrtkOCGAxV5SDABHRHfCuQQ_AUICCgD',
 '/url?q=https://maps.google.com/maps%3Fq%3Dyahoo%2Bfinance%2BTSLA%26prmd%3Dnivsbmtz%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiQ2IrtkOCGAxV5SDABHRHfCuQQiaAMCAkoBA&usg=AOvVaw2hT444SLUebHwoCj5oq_W4',
 '/url?q=/search%3Fq%3Dyahoo%2Bfinance%2BTSLA%26sca_esv%3D66c5d7ec1899268c%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEw

In [6]:
import re
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support','google']
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [7]:
cleaned_urls = strip_unwanted_urls(raw_urls,exclude_list)
cleaned_urls

['https://finance.yahoo.com/news/chinese-evs-are-driving-into-mexico--and-its-starting-to-worry-the-us-210133299.html',
 'https://malaysia.news.yahoo.com/china-tech-etfs-upbeat-momentum-170000754.html',
 'https://finance.yahoo.com/video/tesla-vs-toyota-kevin-paffrath-120012460.html',
 'https://au.news.yahoo.com/opinion-standard-view-london-stock-121348509.html',
 'https://au.finance.yahoo.com/news/cybertruck-buyers-say-theyve-been-told-deliveries-are-paused-due-windshield-wiper-problems-173804018.html',
 'https://malaysia.news.yahoo.com/oracle-shutting-down-advertising-business-120725759.html%3F.tsrc%3Dfp_deeplink',
 'https://finance.yahoo.com/news/elon-musks-44-9b-tesla-215337162.html',
 'https://ca.finance.yahoo.com/news/elon-musks-pay-package-approval-214016988.html',
 'https://uk.movies.yahoo.com/finance/news/next-paramount-skydance-merger-talks-001803498.html',
 'https://malaysia.news.yahoo.com/magna-mga-hits-52-week-190000810.html']

# Retriving Article From a website

In [12]:
# !pip install newspaper3k ####Used to efficiently Retrieve Article from a website
from newspaper import Article

In [13]:
url = 'https://indianexpress.com/article/business/commodities/reliance-buy-russian-oil-from-rosneft-in-rouble-opec-oil-cut-9358229/'
article = Article(url)
article.download()
article.parse()

In [14]:
article.publish_date

datetime.datetime(2024, 5, 29, 8, 6, 34, tzinfo=tzoffset(None, 19800))

In [15]:
article.text

'India’s Reliance Industries, operator of the world’s biggest refining complex, has signed a one-year deal with Russia’s Rosneft to buy at least 3 million barrels of oil a month in roubles, four sources aware of the matter told Reuters.\n\nThe shift to rouble payments follows Russian President Vladimir Putin’s push for Moscow and its trading partners to find alternatives to the Western financial system to facilitate trade despite U.S. and European sanctions.\n\nA term deal with Rosneft also helps privately run Reliance to secure oil at discounted rates at a time when the OPEC+ group of oil producers is expected to extend voluntary supply cuts beyond June.\n\nAdvertisement\n\nThe OPEC+ group comprising the Organisation of the Petroleum Exporting Countries (OPEC) and allies including Russia is due to discuss the output cuts in an online meeting on June 2.\n\nIndia, the world’s third-biggest oil importer and consumer, has become the biggest buyer of seaborne Russian crude since the West h

# Using Text Blob to calculate sentiment Score

In [10]:
# !pip install newspaper3k     ####Used to efficiently Retrieve Article from a website
from newspaper import Article
from textblob import TextBlob
import numpy as np
def calculate_setiment(urls):
    ans = []
    for url in urls:
        article = Article(url)
        article.download()
        article.parse()
        # print(article.text)
        blob = TextBlob(article.text)
        ans.append(blob.sentiment.polarity)
    return np.array(ans)

In [11]:
calculate_setiment(cleaned_urls).mean()

0.06646089714581208

# Using Groq API to access LLAMA3 - 70B model

In [16]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [17]:
import os
from groq import Groq

In [18]:
client = Groq(
    api_key='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',
)

In [19]:
def calculate_setiment_llama(article):
    prompt = f'''Do the sentiment analysis of this article and give out a real value between 0 and 1, 0 showing negative sentiment and 1 showing positive sentiment. Only output real value between 0 and 1.
    Article:
    {article}
    '''
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [20]:
url = 'https://indianexpress.com/article/business/commodities/reliance-buy-russian-oil-from-rosneft-in-rouble-opec-oil-cut-9358229/'
article = Article(url)
article.download()
article.parse()

In [21]:
calculate_setiment_llama(article.text)

'Based on the sentiment analysis of this article, I would give a sentiment score of 0.67.\n\nThe article reports on a deal between Reliance Industries and Rosneft, which is a neutral to positive development. The language used is mostly neutral and factual, with no overtly negative or emotional tone. The article presents the deal as a strategic partnership and quotes Rosneft as saying that India is a strategic partner, which adds a slightly positive tone. The article also mentions that the deal helps Reliance to secure oil at discounted rates, which is a positive development.\n\nHowever, the article also mentions the context of US and European sanctions against Russia, which adds a slightly negative tone. Overall, the sentiment of the article is more positive than negative, but it is still a fairly neutral and factual report.'

In [22]:
##Extracting Sentiment Score from this text
import re
def extract_real_number(text):
    # Define a regex pattern for a real number between 0 and 1
    pattern = r'\b0?\.\d+\b'

    # Search for the pattern in the text
    match = re.search(pattern, text)

    if match:
        # Convert the matched string to a float and return
        return float(match.group())
    else:
        # Return None if no match is found
        return None

number = extract_real_number(calculate_setiment_llama(article.text))
print(number)  # Output should be 0.8


0.67


# Using other sources to retrive News Link


In [23]:
## Using finviz.com
url = 'https://finviz.com/quote.ashx?t=amzn'

In [24]:
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
req = Request(url=url, headers={'user-agent': 'my-app'})
response = urlopen(req)

html = BeautifulSoup(response, features='html.parser')
news_table = html.find(id='news-table')

In [25]:
news_link = []
for row in news_table.findAll('tr'):
    # title = row.a.text
    # news_link.append([date, time, title])
    # news_link.append()
    temp = []
    for x in row.find_all('a', href=True):
        temp.append(x['href'])
    news_link.append(temp)

news_link            #############Try to pick top 15 news because it will be hard to sort on basis of date

[['https://finance.yahoo.com/m/5a5f9003-5a17-3930-91cf-765ecaa18836/6-artficial-intelligence-%28ai%29.html'],
 ['https://finance.yahoo.com/m/9388d37e-88ab-36f7-be95-06b77c959abc/could-this-bull-market-buy.html'],
 ['https://finance.yahoo.com/m/c3922728-02d1-36b9-a9de-79642fa015cc/where-will-amazon-stock-be-in.html'],
 ['https://investorplace.com/2024/06/3-ai-stocks-to-turn-10000-into-1-million-june-2024/'],
 ['https://finance.yahoo.com/m/944e70e8-542f-3c5e-b84e-d0fed2a49b95/3-top-stocks-that.html'],
 [],
 ['https://finance.yahoo.com/m/b3a3d62e-e8a2-3fce-a0c3-6cc058e3162d/a-once-in-a-generation.html'],
 ['https://finance.yahoo.com/m/fd21e48f-6501-3050-806d-a77dcdaf51ad/is-plug-power-stock-a.html'],
 ['https://investorplace.com/2024/06/stock-splits-101-the-5-most-common-questions-answered/'],
 ['https://finance.yahoo.com/m/cf27049b-3a35-3811-8d84-44a4e92bfb60/2-warren-buffett-stocks-to.html'],
 ['https://finance.yahoo.com/m/20bbe145-3966-314b-8d7f-4560fb9306d8/3-reasons-investors-should.